## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [15]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [3]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [5]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [10]:
# Answer below:

yelp['normalized'] = yelp.text.apply(lambda x: normalize(x))

In [11]:
yelp.head()

,text,sentiment,normalized
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [20]:
# Answer below:

from nltk import word_tokenize

words = []
for doc in yelp.normalized:
  for word in word_tokenize(doc):
      words.append(word)

unique = set(words)
nb_unique_words = len(unique)
nb_unique_words

1626

In [21]:
from tensorflow.keras.preprocessing.text import one_hot

In [22]:
encoded_docs = [one_hot(doc, nb_unique_words) for doc in yelp.normalized]
encoded_docs

[[696, 1009, 335],
 [1180, 593, 402],
 [593, 551, 1339, 552],
 [364, 644, 689, 1333, 442, 69, 537, 488, 1009, 974],
 [237, 452, 775, 1003],
 [529, 685, 926, 1504, 546],
 [1029, 1426, 404],
 [1347, 653, 539, 1568, 1559, 696, 97, 1283, 1486, 1193],
 [725, 775, 858],
 [775, 154],
 [1505, 1544],
 [67, 593, 296, 1540],
 [182, 618, 857, 1276, 367, 1384, 460, 588, 162],
 [1304, 616, 1360, 700, 1414, 281, 118],
 [1245, 20, 151, 472, 1327],
 [958, 618, 316, 1560, 676, 962],
 [1619, 488],
 [461, 57, 1553, 1505],
 [335, 593, 429, 1283, 836, 1286, 1152],
 [593, 653, 93],
 [60, 1150],
 [1212, 778],
 [1505, 571, 1073],
 [1568, 857, 1209, 336, 483],
 [810],
 [393, 1388, 1517, 112, 238, 400, 288, 871, 1253, 402],
 [1526, 355, 1474, 524, 1489],
 [1156, 918, 775, 651, 1204, 1095, 1314, 1311],
 [1572,
  521,
  529,
  1212,
  987,
  1260,
  1212,
  1267,
  380,
  1214,
  1591,
  1372,
  653,
  854,
  178],
 [1401, 1231, 1212],
 [571, 705, 653, 1275, 725, 23, 43, 742, 1231],
 [653, 98, 829],
 [603, 335, 14

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [24]:
# Answer below:

from keras.preprocessing.sequence import pad_sequences

ind_vars = pad_sequences(encoded_docs)
ind_vars

array([[   0,    0,    0, ...,  696, 1009,  335],
       [   0,    0,    0, ..., 1180,  593,  402],
       [   0,    0,    0, ...,  551, 1339,  552],
       ...,
       [   0,    0,    0, ...,  593,  296, 1540],
       [   0,    0,    0, ..., 1254, 1271, 1283],
       [   0,    0,    0, ..., 1572,  401,  144]], dtype=int32)

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [45]:
# Answer below:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(ind_vars, np.array(yelp.sentiment), test_size=0.2)


Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [46]:
max_words = np.max(ind_vars)
max_words

1625

In [47]:
X_train.shape

(780, 83)

In [58]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense

model = Sequential()
model.add(Embedding(nb_unique_words, 100, input_length=ind_vars.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 83, 100)           162600    
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 9         
Total params: 167,129
Trainable params: 167,129
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [59]:
# Answer below:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/50
7/7 [==============================] - 2s 102ms/step - loss: 0.6909 - accuracy: 0.5448 - val_loss: 0.6829 - val_accuracy: 0.5765
Epoch 2/50
7/7 [==============================] - 0s 69ms/step - loss: 0.6460 - accuracy: 0.6790 - val_loss: 0.6767 - val_accuracy: 0.5816
Epoch 3/50
7/7 [==============================] - 0s 68ms/step - loss: 0.6003 - accuracy: 0.8597 - val_loss: 0.6715 - val_accuracy: 0.5918
Epoch 4/50
7/7 [==============================] - 0s 71ms/step - loss: 0.5488 - accuracy: 0.9038 - val_loss: 0.6649 - val_accuracy: 0.5969
Epoch 5/50
7/7 [==============================] - 0s 69ms/step - loss: 0.4869 - accuracy: 0.9298 - val_loss: 0.6586 - val_accuracy: 0.6224
Epoch 6/50
7/7 [==============================] - 0s 71ms/step - loss: 0.4168 - accuracy: 0.9561 - val_loss: 0.6526 - val_accuracy: 0.6020
Epoch 7/50
7/7 [==============================] - 0s 65ms/step - loss: 0.3533 - accuracy: 0.9596 - val_loss: 0.6429 - val_accuracy: 0.6276
Epoch 8/50
7/7 [==========